In [1]:
import os
from os.path import join
import numpy as np
from cell_mrcnn.utils import get_cell_mrcnn_path_from_config_file
from matplotlib import pyplot as plt
from imgaug import augmenters as iaa
import tensorflow as tf

# Import Mask RCNN
from cell_mrcnn.cell import CellConfig, CellDataset
from cell_mrcnn import model as modellib, utils


# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs
root = get_cell_mrcnn_path_from_config_file()
DEFAULT_LOGS_DIR = join(root, "logs")
dataset_dir = join(root, 'data/annotated_datasets')

def train(model):
    """Train the model."""
    dataset_path = join(dataset_dir, '2020_12_15_22_01_08/')
    # Training dataset.
    dataset_train = CellDataset()
    dataset_train.load_cell(dataset_path, "train")
    dataset_train.prepare()

    # Validation dataset
    dataset_val = CellDataset()
    dataset_val.load_cell(dataset_path, "val")
    dataset_val.prepare()

    # Image augmentation
    # http://imgaug.readthedocs.io/en/latest/source/augmenters.html
    augmentation = iaa.SomeOf((0, 2), [
        iaa.Fliplr(0.5),
        iaa.Flipud(0.5),
        iaa.OneOf([iaa.Affine(rotate=90),
                   iaa.Affine(rotate=180),
                   iaa.Affine(rotate=270)]),
        iaa.GaussianBlur(sigma=(0.0, 1.0)),
        iaa.Add((-10, 10)),
        iaa.Multiply((0.8, 1.5))
    ])

    print("Train all layers")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=10,
                augmentation=augmentation,
                layers='all')

In [2]:
config = CellConfig()
DEVICE = "/gpu:0"  # /cpu:0 or /gpu:0
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="training", config=config,
                              model_dir=DEFAULT_LOGS_DIR)

# load the last model trained
# weights_path = model.find_last()
# model.load_weights(weights_path, by_name=True)







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
box_ind is deprecated, use box_indices instead


In [ ]:
train(model)

Train all layers


/home/miska/anaconda3/envs/cell_mrcnn/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)



Starting at epoch 0. LR=0.001

Checkpoint Path: /media/miska/Data/cell_mrcnn_data/logs/cell20201216T2015/mask_rcnn_cell_{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2b          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res3a_branch2a         (Conv2D)
bn3a_branch2a          (BatchNorm)
res3a_branch2b         (Conv2D)
res3a_branch1          (Conv2D)
bn3a_branch2b          (BatchNorm)
bn3a_branch1           (BatchNorm)
res3b_branch2a         (Conv2D)
bn3b_branch2a          (BatchNorm)
res3b_branch2b         (Conv2D)
bn3b_branch2b          (BatchNorm)
res4a_branch2a         (Conv2D)
bn4a_branch2a          (BatchNorm)
res4a_branch2b         (Conv2D)
re

/home/miska/anaconda3/envs/cell_mrcnn/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/miska/anaconda3/envs/cell_mrcnn/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/miska/anaconda3/envs/cell_mrcnn/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


/home/miska/anaconda3/envs/cell_mrcnn/lib/python3.7/site-packages/keras/engine/training_generator.py:47: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'







Epoch 1/10
200/200 [==============================] - 349s 2s/step - loss: 1.9231 - rpn_class_loss: 0.2686 - rpn_bbox_loss: 0.3619 - mrcnn_class_loss: 0.2258 - mrcnn_bbox_loss: 0.5080 - mrcnn_mask_loss: 0.5589 - val_loss: 3.5527 - val_rpn_class_loss: 0.9409 - val_rpn_bbox_loss: 0.9176 - val_mrcnn_class_loss: 0.3902 - val_mrcnn_bbox_loss: 0.6381 - val_mrcnn_mask_loss: 0.6659

Epoch 2/10
  9/200 [>.............................] - ETA: 2:49 - loss: 1.4583 - rpn_class_loss: 0.1146 - rpn_bbox_loss: 0.2192 - mrcnn_class_loss: 0.1901 - mrcnn_bbox_loss: 0.3464 - mrcnn_mask_loss: 0.5880

Process ForkPoolWorker-4:
Process ForkPoolWorker-2:
Process ForkPoolWorker-5:
Process ForkPoolWorker-1:
Process ForkPoolWorker-6:
Process ForkPoolWorker-8:
Process ForkPoolWorker-7:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/miska/anaconda3/envs/cell_mrcnn/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/miska/anaconda3/envs/cell_mrcnn/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/miska/anaconda3/envs/cell_mrcnn/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/miska/anaconda3/envs/cell_mrcnn/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/miska/anaconda3/envs/cell_mrcnn/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/miska/anaconda3/en